<picture>
<a href="https://sambanova.ai/"\>
<source media="(prefers-color-scheme: dark)" srcset="../../images/SambaNova-light-logo-1.png" height="60">
<img alt="SambaNova logo" src="../../images/SambaNova-dark-logo-1.png" height="60">
</picture>
</a>

# Semantic Kernel Example

Code adapted from [here](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started_with_agents/step3_chat.py). The following sample demonstrates how to create a simple, agent group chat that utilizes An Art Director Chat Completion Agent along with a Copy Writer Chat Completion Agent to complete a task.   

You will need to create a [SambaNova Cloud API key](https://cloud.sambanova.ai/apis) and add to the `.env` file in the repo the following variables needed:
```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="<<YOUR_API_KEY>>"
OPENAI_BASE_URL="https://api.sambanova.ai/v1/"
OPENAI_CHAT_MODEL_ID="Meta-Llama-3.1-70B-Instruct"
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
OPENAI_ORG_ID=""
```
Please review [this notebook](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started/00-getting-started.ipynb) if you have any questions related to these env variables. In this example, we're levaraging from the fact that SambaNova Cloud is OpenAI compatible.

In [ ]:
import sys
sys.path.append('../../')
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import DefaultTerminationStrategy
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent

from dotenv import load_dotenv 
load_dotenv(override=True)

In [3]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(OpenAIChatCompletion(service_id=service_id))
    return kernel

In [4]:
class ApprovalTerminationStrategy(DefaultTerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [5]:
async def main():
    
    # Art director agent
    REVIEWER_NAME = "ArtDirector"
    REVIEWER_INSTRUCTIONS = """
    You are an art director who has opinions about copywriting born of a love for David Ogilvy.
    The goal is to determine if the given copy is acceptable to print.
    If so, state that it is approved.
    If not, provide insight on how to refine suggested copy without example.
    """
    agent_reviewer = ChatCompletionAgent(
        service_id="artdirector",
        kernel=_create_kernel_with_chat_completion("artdirector"),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    # Copywriter agent
    COPYWRITER_NAME = "CopyWriter"
    COPYWRITER_INSTRUCTIONS = """
    You are a copywriter with ten years of experience and are known for brevity and a dry humor.
    The goal is to refine and decide on the single best copy as an expert in the field.
    Only provide a single proposal per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """
    agent_writer = ChatCompletionAgent(
        service_id="copywriter",
        kernel=_create_kernel_with_chat_completion("copywriter"),
        name=COPYWRITER_NAME,
        instructions=COPYWRITER_INSTRUCTIONS,
    )

    # Group chat
    group_chat = AgentGroupChat(
        agents=[
            agent_writer,
            agent_reviewer,
        ],
        termination_strategy=ApprovalTerminationStrategy(
            agents=[agent_reviewer],
            maximum_iterations=10,
        ),
    )

    # Start the group chat and wait for it to complete
    input = "a slogan for a new line of electric cars."

    await group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
    print(f"# User: '{input}'")

    async for content in group_chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {group_chat.is_complete}")


In [ ]:
await main()